# LSH and Document Search

Implementing a more efficient version of k-nearest neighbors using locality sensitive hashing (LSH) and then applying this to document search. Explicitly:

- Processing the tweets and representing each tweet as a vector (representing a document with a vector embedding).
- Using locality sensitive hashing and k nearest neighbors to find tweets that are similar to a given tweet.

## 1. Initialization

#### Importing packages

In [1]:
import nltk
import re
import string
import pickle
import numpy as np
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

#### Downloading twitter samples and stopwords from nltk

In [2]:
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahmoud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Mahmoud\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

#### Getting the positive and negative tweets

In [3]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
all_tweets = all_positive_tweets + all_negative_tweets

Loading english embeddings.  
For more details, refer to: https://github.com/mahmoudjs14/NLP/blob/master/word2vec/NaiveMachineTranslation.ipynb

In [4]:
en_embeddings_subset = pickle.load(open("../word2vec/data/en_embeddings.p", "rb"))

#### Processing tweets

**Defining process_tweet function**

**Inputs** :  
- *tweet*: a string containing a tweet

**Output** :  
- *tweets_clean*: a list of words containing the processed tweet

In [5]:
def process_tweet(tweet):

    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    
    # removing stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # removing old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # removing hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # removing hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # removing stopwords
            word not in string.punctuation):  # removing punctuation

            stem_word = stemmer.stem(word)
            tweets_clean.append(stem_word)

    return tweets_clean

**Defining cosine_similarity function**

**Inputs**:
- *A*: a numpy array which corresponds to a word vector
- *B*: A numpy array which corresponds to a word vector

**Outputs**:
- *cos*: numerical number representing the cosine similarity between A and B.

In [6]:
def cosine_similarity(A, B):

    cos = -10
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)

    return cos

**Defining nearest_neighbor function**

**Inputs** :  
*v*: the vector we are going find the nearest neighbor for
*candidates*: a set of vectors where we will find the neighbors  
*k*: top k nearest neighbors to find

**Outputs** :  
*k_idx*: the indices of the top k closest vectors in sorted form

In [7]:
def nearest_neighbor(v, candidates, k=1):

    similarity_l = []

    for row in candidates:
        cos_similarity = cosine_similarity(v,row)
        similarity_l.append(cos_similarity)
        
    sorted_ids = np.argsort(similarity_l)

    k_idx = sorted_ids[-k:]
    
    return k_idx

## 2. Getting the Document Embeddings

**Defining get_document_embedding function**

**Inputs** :  
- *tweet*: a string  
- *en_embeddings*: a dictionary of word embeddings

**Outputs** :  
- *doc_embedding*: sum of all word embeddings in the tweet

In [8]:
def get_document_embedding(tweet, en_embeddings): 

    doc_embedding = np.zeros(300)

    processed_doc = process_tweet(tweet)
    for word in processed_doc:
        doc_embedding += en_embeddings.get(word,0)

    return doc_embedding

#### Storing all document vectors into a dictionary

**Defining get_document_vecs function**

**Inputs** :  
- *all_docs*: list of strings - all tweets in our dataset.  
- *en_embeddings*: dictionary with words as the keys and their embeddings as the values.

**Outputs** :
- *document_vec_matrix*: matrix of tweet embeddings.
- *ind2Doc_dict*: dictionary with indices of tweets in vecs as keys and their embeddings as the values.

In [9]:
def get_document_vecs(all_docs, en_embeddings):

    ind2Doc_dict = {}

    document_vec_l = []

    for i, doc in enumerate(all_docs):
        doc_embedding = get_document_embedding(doc, en_embeddings)
        ind2Doc_dict[i] = doc_embedding
        document_vec_l.append(doc_embedding)

    document_vec_matrix = np.vstack(document_vec_l)

    return document_vec_matrix, ind2Doc_dict

In [10]:
document_vecs, ind2Tweet = get_document_vecs(all_tweets, en_embeddings_subset)
print(f"length of dictionary {len(ind2Tweet)}")
print(f"shape of document_vecs {document_vecs.shape}")

length of dictionary 10000
shape of document_vecs (10000, 300)


## 2. Looking up the tweets

Inputing a tweet, and use cosine similarity to see which tweet in our corpus is similar to this tweet.

In [11]:
my_tweet = 'i like listening to rock music!'
process_tweet(my_tweet)
tweet_embedding = get_document_embedding(my_tweet, en_embeddings_subset)

In [12]:
idx = np.argmax(cosine_similarity(document_vecs, tweet_embedding))
print(all_tweets[idx])

I honestly had a blast getting shitfaced with the chat and with all you kind souls donating over 70$ tonight, singing disney songs was :D


## 3. Finding the most similar tweets with LSH

Implementing locality sensitive hashing (LSH) to identify the most similar tweet.

In [13]:
N_VECS = len(all_tweets)
N_DIMS = len(ind2Tweet[1])
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

Number of vectors is 10000 and each has 300 dimensions.


**Choosing the number of planes**  
Each plane divides the space to  2  parts, so  𝑛  planes divide the space into  2𝑛  hash buckets.
We want to organize 10,000 document vectors into buckets so that every bucket has about 16 vectors, for that we need 10000/16=625 buckets.
We're interested in 𝑛 , number of planes, so that 2𝑛=625. Now, we can calculate 𝑛=log2(625)=9.29≈10.

In [14]:
N_PLANES = 10
N_UNIVERSES = 25 # Number of times to repeat the hashing to improve the search

## 4. Getting the hash number for a vector

#### Implementing hash buckets

In [15]:
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES)) for _ in range(N_UNIVERSES)]

#### Creating a hash for a vector

**Defining hash_value_of_vector function**

**Inputs**:
- *v*: vector of tweet. It's dimension is (1, N_DIMS)
- *planes*: matrix of dimension (N_DIMS, N_PLANES) - the set of planes that divide up the region


**Outputs**:
- *res*: a number which is used as a hash for the vector

In [16]:
def hash_value_of_vector(v, planes):

    dot_product = np.dot(v,planes)
    sign_of_dot_product = np.sign(dot_product)

    h = (sign_of_dot_product >= 0)
    h = np.squeeze(h)

    hash_value = 0

    n_planes = planes.shape[1]
    for i in range(n_planes):
        hash_value += (2**i)*h[i]

    hash_value = int(hash_value)

    return hash_value

## 5. Creating a hash table

**Defining make_hash_table function**

**Inputs**:
- *vecs*: list of vectors to be hashed.
- *planes*: the matrix of planes in a single "universe", with shape (embedding dimensions, number of planes).

**Outputs**:
- *hash_table*: dictionary - keys are hashes, values are lists of vectors (hash buckets)
- *id_table*: dictionary - keys are hashes, values are list of vectors id's (it's used to know which tweet corresponds to the hashed vector)

In [17]:
def make_hash_table(vecs, planes):

    num_of_planes = np.shape(planes)[1]
    num_buckets = 2**num_of_planes

    hash_table = {i:[] for i in range(num_buckets)}
    id_table = {i:[] for i in range(num_buckets)}

    for i, v in enumerate(vecs):
        h = hash_value_of_vector(v, planes)
        hash_table[h].append(v)
        id_table[h].append(i)

    return hash_table, id_table

## 6. Creating all hash tables

In [18]:
hash_tables = []
id_tables = []
for universe_id in range(N_UNIVERSES):
    print('working on hash universe #:', universe_id)
    planes = planes_l[universe_id]
    hash_table, id_table = make_hash_table(document_vecs, planes)
    hash_tables.append(hash_table)
    id_tables.append(id_table)

working on hash universe #: 0
working on hash universe #: 1
working on hash universe #: 2
working on hash universe #: 3
working on hash universe #: 4
working on hash universe #: 5
working on hash universe #: 6
working on hash universe #: 7
working on hash universe #: 8
working on hash universe #: 9
working on hash universe #: 10
working on hash universe #: 11
working on hash universe #: 12
working on hash universe #: 13
working on hash universe #: 14
working on hash universe #: 15
working on hash universe #: 16
working on hash universe #: 17
working on hash universe #: 18
working on hash universe #: 19
working on hash universe #: 20
working on hash universe #: 21
working on hash universe #: 22
working on hash universe #: 23
working on hash universe #: 24


#### Approximate K-NN

**Defining approximate_knn function**

**Inputs**:
- *v*: the document vector for the tweet we are willing to find neighbors.
- *planes_l*: the list of planes (the global variable created earlier).
- *k*: the number of nearest neighbors to search for.
- *num_universes_to_use*: N_UNIVERSES

**Outputs**:
- *nearest_neighbor_ids*: the index of k nearest documents to the given document.

In [19]:
def approximate_knn(v, planes_l, k=1, num_universes_to_use=N_UNIVERSES):

    assert num_universes_to_use <= N_UNIVERSES

    vecs_to_consider_l = list()
    ids_to_consider_l = list()
    ids_to_consider_set = set()

    for universe_id in range(num_universes_to_use):
        planes = planes_l[universe_id]
        hash_value = hash_value_of_vector(v, planes)
        hash_table = hash_tables[universe_id]
        document_vectors_l = hash_table[hash_value]
        id_table = id_tables[universe_id]
        new_ids_to_consider = id_table[hash_value]

        for i, new_id in enumerate(new_ids_to_consider):

            if new_id not in ids_to_consider_set:
                document_vector_at_i = document_vectors_l[i]
                vecs_to_consider_l.append(document_vector_at_i)
                ids_to_consider_l.append(new_id)
                ids_to_consider_set.add(new_id)

    print("Fast considering %d vecs" % len(vecs_to_consider_l))

    vecs_to_consider_arr = np.array(vecs_to_consider_l)

    nearest_neighbor_idx_l = nearest_neighbor(v, vecs_to_consider_arr, k=k)

    nearest_neighbor_ids = [ids_to_consider_l[idx] for idx in nearest_neighbor_idx_l]

    return nearest_neighbor_ids

#### Using LSH to find 3 nearest neighbors to my sample tweet

In [20]:
nearest_neighbor_ids = approximate_knn(tweet_embedding, planes_l, k=3, num_universes_to_use=5)

Fast considering 68 vecs


In [21]:
print(f"Document contents: {my_tweet}")
print("")

for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {all_tweets[neighbor_id]}")

Document contents: i like listening to rock music!

Nearest neighbor at document id 8930
document contents: IM LISTENING TO EVERYTHING ABOUT YOU :((
Nearest neighbor at document id 1934
document contents: @composurs listen :-) buddy :-) u wanna play it like this :-)
Nearest neighbor at document id 634
document contents: @NativeMusic49 but I listen to jazz in my truck. :)
